In [ ]:
# install all packages to access .env and .local.env, and then access aws bedrock 

%pip install python-dotenv boto3 awscli Flask

In [3]:
import os
from dotenv import load_dotenv
import json

load_dotenv(dotenv_path="../.env")

load_dotenv(dotenv_path="../.local.env")


import boto3

# get the aws credentials from the .local.env file in ../.local.env
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
accept = "application/json"
contentType = "application/json"


from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")


# Set the model ID, e.g., Mistral Large.
model_id = "mistral.mistral-7b-instruct-v0:2"

# Define the prompt for the model.
prompt = "Show me the best way to program a python websocket server using flask that can handle a stream from boto3.client('bedrock-runtime')"

# Embed the prompt in Mistral's instruction format.
formatted_prompt = f"<s>[INST] {prompt} [/INST]"

# Format the request payload using the model's native structure.
native_request = {
    "prompt": formatted_prompt,
    "max_tokens": 3000,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(native_request)
def invoke_model(client, model_id, request, stream=False):
    try:
        if stream:
            # Invoke the model with the request and stream the response.
            streaming_response = client.invoke_model_with_response_stream(
                modelId=model_id, body=request
            )

            # Extract and print the response text in real-time.
            for event in streaming_response["body"]:
                chunk = json.loads(event["chunk"]["bytes"])
                if "outputs" in chunk:
                    print(chunk["outputs"][0].get("text"), end="")
        else:
            # Invoke the model with the request.
            response = client.invoke_model(modelId=model_id, body=request)

            # Decode the response body.
            model_response = json.loads(response["body"].read())

            # Extract and print the response text.
            response_text = model_response["outputs"][0]["text"]
            print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

# Invoke the model and print the response.
invoke_model(client, model_id, request, True)

 To create a Python WebSocket server using Flask that can handle a stream from `boto3.client('bedrock-runtime')`, you'll first need to install Flask and the Flask-SocketIO library. You can do this using pip:

```bash
pip install Flask Flask-SocketIO eventlet boto3
```

Here's a basic example of a Flask WebSocket server that can handle a stream from AWS Bedrock:

```python
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import boto3
import json

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

bedrock_client = boto3.client('bedrock-runtime')

def handle_stream(event, context):
    data = json.loads(event['Data'])
    emit('message', data, broadcast=True)

@socketio.on('connect')
def test_connect():
    print('Client connected')
    emit('message', {'data': 'Connected!'})

@socketio.on('disconnect')
def test_disconnect():
    print('Client disconnected')

@app.route('/')
def index():
    return render_template('index.